In [84]:
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px

import pandas as pd
import plotly.graph_objects as go
import os
import kaleido
import re

In [3]:
path_report = 'C:/Models/SATIMGE_veda/REPORT_00.csv'
path_setsandmaps = 'C:/Models/SATIMGE_veda/setsandmaps/setsandmaps.xlsm'


# --- Create output folder ---
output_dir = "C:/Models/NDC_Figures"
os.makedirs(output_dir, exist_ok=True)


## Load dataset

In [4]:
# Read in the DataFrames
report_df = pd.read_csv(path_report, low_memory=False)#
mapPRC_df = pd.read_excel(path_setsandmaps, sheet_name="mapPRC")
mapCOM_df = pd.read_excel(path_setsandmaps, sheet_name="mapCOM")


## Add setsandmaps

In [5]:

# Replace 'Eps' values with 0
report_df['SATIMGE'] = report_df['SATIMGE'].replace('Eps', 0)
report_df['SATIMGE'] = report_df['SATIMGE'].astype(float)

# Merge the DataFrames
merged_df = report_df.merge(mapPRC_df, on='Process', how='left')
merged_df = merged_df.merge(mapCOM_df, on='Commodity', how='left')

# Reset index
merged_df.reset_index(drop=True, inplace=True)

merged_df.head()

,Process,Commodity,Year,Scenario,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name
0,SATIMMarginals,CF4,2021,NDC_CPP4-08-RG,Marginals,-2999.004873,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4
1,SATIMMarginals,CF4,2021,NDC_CPP4-085-RG,Marginals,-1819.246552,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4
2,SATIMMarginals,CF4,2021,NDC_CPP4-09-RG,Marginals,-858.259367,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4
3,SATIMMarginals,CF4,2021,NDC_CPP3-075-RG,Marginals,-4677.926676,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4
4,SATIMMarginals,CF4,2021,NDC_CPP3-08-RG,Marginals,-3037.996489,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4


# ADD CO2eq to the data

In [6]:
# GWP values (AR5 100-year)
gwp_factors = {
    'CO2': 1,
    'CO2eq': 1,
    'CH4': 28,
    'N2O': 265,
    'CF4': 6630,
    'C2F6': 11100
}

# Ensure SATIMGE is numeric
merged_df['SATIMGE'] = pd.to_numeric(merged_df['SATIMGE'], errors='coerce')

# Apply GWP conversion
merged_df['CO2eq'] = merged_df.apply(
    lambda row: row['SATIMGE'] * gwp_factors.get(row['Indicator'], 0),
    axis=1
)



# Define other mappings

In [9]:
# Define ordered list of (substring, ScenarioFamily) mappings
scenario_mapping = [
    ('CPP4', 'CPP4 Variant'),  # must come before exact 'CPP4' to catch variants like CPP4EK
    ('CPP4', 'CPP4'),          # exact match
    ('CPP1', 'CPP1'),
    ('CPP2', 'CPP2'),
    ('CPP3', 'CPP3'),
    ('HCARB', 'High Carbon'),
    ('LCARB', 'Low Carbon'),
    ('BASE', 'BASE')
]


In [8]:
# Function to assign ScenarioFamily based on rules
def assign_scenario_family(scenario):
    # First handle exact match for 'CPP4'
    if scenario.strip() == 'CPP4':
        return 'CPP4'
    for key, value in scenario_mapping:
        if key in scenario:
            return value
    return 'Other'

In [98]:
# add mapping for economic growth rates

def map_economic_growth(scenario):
    if '-RG' in scenario:
        return 'Reference'
    elif '-LG' in scenario:
        return 'Low'
    elif '-HG' in scenario:
        return 'High'
    else:
        return 'Unknown'


# carbon budget mapping
carbonbudget_map = {
'075':7.5,
'0775':7.75,    
'08':8,
'8':8,
'0825':8.25,
'085':8.5,
'0875':8.75,
'09':9,
'0925':9.25,
'095':9.5,
'0975':9.75,
'10':10,
'1025':10.25,
'105':10.5
}


## Cut down dataset to just 2024 to 2035

In [100]:
yr_start = 2024
yr_stop = 2035

df = merged_df[merged_df['Year'].isin(range(yr_start,yr_stop+1))]


# Apply mappings

In [ ]:
# apply mappings to the main dataset
# carbon budgets
df['number_str'] = df['Scenario'].str.extract(r'-(\d+)-', expand=False)
df['CarbonBudget'] = df['number_str'].map(carbonbudget_map).fillna("NoBudget")
df.drop(columns=['number_str'], inplace=True) #remove this column


#scenraio family
df['ScenarioFamily'] = df['Scenario'].apply(assign_scenario_family)

# add economic growth rates
df['EconomicGrowth'] = df['Scenario'].apply(map_economic_growth) 

#show
df.head()

C:\Users\01425453\AppData\Local\Temp\ipykernel_47340\3604327681.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\01425453\AppData\Local\Temp\ipykernel_47340\3604327681.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\01425453\AppData\Local\Temp\ipykernel_47340\3604327681.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,Process,Commodity,Year,Scenario,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,...,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,CO2eq,number_str,budget,ScenarioFamily,EconomicGrowth
138,SATIMMarginals,CF4,2024,NDC_CPP4-08-RG,Marginals,-3798.909553,All,All,All,SATIMMarginals Annual,...,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,08,8.0,CPP4 Variant,Reference
139,SATIMMarginals,CF4,2024,NDC_CPP4-085-RG,Marginals,-2304.482119,All,All,All,SATIMMarginals Annual,...,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,085,8.5,CPP4 Variant,Reference
140,SATIMMarginals,CF4,2024,NDC_CPP4-09-RG,Marginals,-1087.177196,All,All,All,SATIMMarginals Annual,...,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,09,9.0,CPP4 Variant,Reference
141,SATIMMarginals,CF4,2024,NDC_CPP3-075-RG,Marginals,-5925.639034,All,All,All,SATIMMarginals Annual,...,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,075,7.5,CPP3,Reference
142,SATIMMarginals,CF4,2024,NDC_CPP3-08-RG,Marginals,-3848.301145,All,All,All,SATIMMarginals Annual,...,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,08,8.0,CPP3,Reference


In [88]:
df.head()


,Process,Commodity,Year,Scenario,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,...,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,CO2eq,ScenarioFamily,CarbonBudget,EconomicGrowth
138,SATIMMarginals,CF4,2024,NDC_CPP4-08-RG,Marginals,-3798.909553,All,All,All,SATIMMarginals Annual,...,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP4 Variant,NaN,Reference
139,SATIMMarginals,CF4,2024,NDC_CPP4-085-RG,Marginals,-2304.482119,All,All,All,SATIMMarginals Annual,...,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP4 Variant,NaN,Reference
140,SATIMMarginals,CF4,2024,NDC_CPP4-09-RG,Marginals,-1087.177196,All,All,All,SATIMMarginals Annual,...,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP4 Variant,9.0,Reference
141,SATIMMarginals,CF4,2024,NDC_CPP3-075-RG,Marginals,-5925.639034,All,All,All,SATIMMarginals Annual,...,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP3,NaN,Reference
142,SATIMMarginals,CF4,2024,NDC_CPP3-08-RG,Marginals,-3848.301145,All,All,All,SATIMMarginals Annual,...,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP3,NaN,Reference


# MAKE CHARTS

### Define common layout

In [34]:
# --- Common layout settings ---
def apply_common_layout(fig, title):
    fig.update_layout(
        title=title,
        template="simple_white",
        font=dict(family="Arial", size=14),
        margin=dict(l=60, r=30, t=60, b=60),
        showlegend=False,
        xaxis=dict(
            tickmode='linear',
            tick0=df['Year'].min(),
            dtick=1,
            range=[df['Year'].min(), df['Year'].max()]
        )
    )
    return fig


## Figure 1: line chart of all scenarios

In [16]:
import plotly.express as px

In [109]:
# --- Fig 1: Line chart of total emissions by scenario ---
fig1_data = df.groupby(['ScenarioFamily', 'Scenario', 'Year'])['CO2eq'].sum().reset_index()
#cnvert to Mt

fig1_data['CO2eq'] = fig1_data['CO2eq']*0.001

In [ ]:
fig1 = go.Figure()

for scenario in fig1_data['Scenario'].unique():
    subset = fig1_data[fig1_data['Scenario'] == scenario]
    fig1.add_trace(go.Scatter(
        x=subset['Year'], y=subset['CO2eq'], mode='lines',
        line=dict(width=1.5, color='gray'),
        showlegend=False
    ))

fig1 = apply_common_layout(fig1, "")



fig1.write_image(f"{output_dir}/fig1_total_emissions_by_scenario.jpeg")
fig1_data.to_csv(f"{output_dir}/fig1_data.csv", index=False)
fig1.show()


## figure 2: Shaded emissions chart for all scenario families

### Aggregate CO2 values

In [29]:

scenario_year_emissions = df
# Sum over all lower-level categories to Scenario-Year level
scenario_year_emissions = (
    df.groupby(['Scenario', 'ScenarioFamily','Year'])['CO2eq']
    .sum()
    .reset_index(name='CO2eq_Total')
)

REMOVE LCARB because we dont need to show it (?)

In [30]:
scenario_year_emissions = scenario_year_emissions[~scenario_year_emissions['ScenarioFamily'].isin(['Low Carbon'])]

### Get aggregate Stats for Shaded Bands (min, max, mean)

In [31]:
## group CPP together

# Group into broader ScenarioGroup
scenario_year_emissions['ScenarioGroup'] = scenario_year_emissions['ScenarioFamily'].apply(
    lambda x: 'CPP' if x.startswith('CPP') else x
)


# Compute min, max, and mean across scenarios in a group
group_stats = (
    scenario_year_emissions
    .groupby(['ScenarioGroup', 'Year'])['CO2eq_Total']
    .agg(['min', 'max', 'mean'])
    .reset_index()
    .rename(columns={'min': 'Emissions_min', 'max': 'Emissions_max', 'mean': 'Emissions_mean'})
)


SHOW THE NUMBERS

In [32]:
group_stats[group_stats['Year'] == 2024]

,ScenarioGroup,Year,Emissions_min,Emissions_max,Emissions_mean
0,BASE,2024,438233.797424,442284.399680,440314.869676
12,CPP,2024,435019.953470,438805.487438,437852.966901
24,High Carbon,2024,440856.216922,443736.935734,441372.908136


In [104]:
group_colors = {
    'CPP': 'rgba(255, 255, 0, 0.2)',        # blue with 20% opacity
    'BASE': 'rgba(0, 0, 255, 0.2)',         # yellow
    'High Carbon': 'rgba(255, 0, 0, 0.2)',# red
    'Low Carbon': 'rgba(0, 128, 0, 0.2)', # green
    'Other': 'rgba(128, 128, 128, 0.2)'   # gray
}

line_colors = {
    'CPP': 'Yellow',
    'BASE': 'Blue',
    'High Carbon': 'red',
    'Low Carbon': 'green',
    'Other': 'gray'
}

fig = go.Figure()


for group in group_stats['ScenarioGroup'].unique():
    data = group_stats[group_stats['ScenarioGroup'] == group]
    fill_color = group_colors.get(group, 'rgba(128,128,128,0.9)')
    line_color = line_colors.get(group, 'gray')

    # Shaded band
    fig.add_trace(go.Scatter(
        x=pd.concat([data['Year'], data['Year'][::-1]]),
        y=pd.concat([data['Emissions_min'], data['Emissions_max'][::-1]]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(255,255,255,0)'),  # no border
        name=f'{group} Range',
        showlegend=True
    ))

    # Mean line
    fig.add_trace(go.Scatter(
        x=data['Year'],
        y=data['Emissions_mean'],
        mode='lines',
        line=dict(color=line_color, width=2),
        name=f'{group} Mean'
    ))


fig.update_layout(
    title='Total CO₂ Emissions by Scenario Group (Shaded Bands)',
    xaxis_title='Year',
    yaxis_title='CO₂ Emissions (kt)',
    template='simple_white',
    font=dict(family='Arial', size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=60, r=30, t=60, b=60)
)


#Write to file
fig.write_image(f"{output_dir}/fig2_shadedbands.jpeg")
group_stats.to_csv(f"{output_dir}/fig2_data.csv", index=False)
fig.show()

## Figure 3: change in emissions to 2035 by sector

In [ ]:
# --- Fig 3: NDC_BASE-RG aggregated and sorted sector groups for 2024 and 2035 ---

# Step 1: Filter and group sectors
subset_df = df[
    (df['CO2eq'] != 0.0) &
    (df['Scenario'] == 'NDC_BASE-RG') &
    (df['Year'].isin([2024, 2035]))
].copy()

def map_sector_group(sector):
    if sector in ['Industry', 'Process emissions']:
        return 'Industry'
    elif sector == 'Power':
        return 'Power'
    elif sector == 'Transport':
        return 'Transport'
    elif sector == 'Refineries':
        return 'Refineries'
    else:
        return 'All others'

subset_df['SectorGroup'] = subset_df['Sector'].apply(map_sector_group)

# Step 2: Aggregate emissions
fig3_data = subset_df.groupby(['Year', 'SectorGroup'])['CO2eq'].sum().reset_index()

# Step 3: Calculate differences and formatted labels
pivot_df = fig3_data.pivot(index='SectorGroup', columns='Year', values='CO2eq').fillna(0)
pivot_df['diff'] = pivot_df[2035] - pivot_df[2024]
pivot_df['label'] = pivot_df.index + ' (' + pivot_df['diff'].round(0).astype(int).astype(str) + ')'
label_map = pivot_df['label'].to_dict()

# Step 4: Map new labels to fig3_data
fig3_data['SectorLabel'] = fig3_data['SectorGroup'].map(label_map)

# Step 5: Sort order
ordered_labels = fig3_data[fig3_data['Year'] == 2035][['SectorGroup', 'SectorLabel']]
ordered_labels['abs_diff'] = ordered_labels['SectorGroup'].map(pivot_df['diff'].abs())
ordered_labels = ordered_labels.drop_duplicates().sort_values('abs_diff')
ordered_label_list = ordered_labels['SectorLabel'].tolist()

# Step 6: Plot
fig3 = go.Figure()

# 2024: red circle
subset_2024 = fig3_data[fig3_data['Year'] == 2024]
fig3.add_trace(go.Scatter(
    x=subset_2024['CO2eq'],
    y=subset_2024['SectorLabel'],
    mode='markers',
    name='2024',
    marker=dict(size=10, color='red', symbol='circle')
))

# 2035: blue X
subset_2035 = fig3_data[fig3_data['Year'] == 2035]
fig3.add_trace(go.Scatter(
    x=subset_2035['CO2eq'],
    y=subset_2035['SectorLabel'],
    mode='markers',
    name='2035',
    marker=dict(size=10, color='blue', symbol='x')
))

# Step 7: Layout
fig3.update_layout(
    title="Fig 3: NDC_BASE-RG Emissions by Aggregate Sector Group (Sorted by Δ 2035-2024)",
    xaxis_title="CO₂eq (kt)",
    yaxis_title="Sector Group (Δ 2035–2024)",
    yaxis=dict(categoryorder='array', categoryarray=ordered_label_list),
    template="simple_white",
    font=dict(family="Arial", size=14),
    margin=dict(l=60, r=30, t=60, b=100),
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5)
)

fig3.show()
fig3.write_image(f"{output_dir}/fig3_ndc_emissions_by_sector.jpeg")
fig3_data.to_csv(f"{output_dir}/fig3_data.csv", index=False)

### Figure 3b: as a bar plot

In [ ]:
# --- Fig 3: NDC_BASE-RG aggregated and sorted sector groups as bar chart ---

# Step 1: Filter and group sectors
subset_df = df[
    (df['CO2eq'] != 0.0) &
    (df['Scenario'] == 'NDC_BASE-RG') &
    (df['Year'].isin([2024, 2035]))
].copy()

def map_sector_group(sector):
    if sector in ['Industry', 'Process emissions']:
        return 'Industry'
    elif sector == 'Power':
        return 'Power'
    elif sector == 'Transport':
        return 'Transport'
    elif sector == 'Refineries':
        return 'Refineries'
    else:
        return 'All others'

subset_df['SectorGroup'] = subset_df['Sector'].apply(map_sector_group)

# Step 2: Aggregate emissions
fig3_data = subset_df.groupby(['Year', 'SectorGroup'])['CO2eq'].sum().reset_index()

# Step 3: Calculate differences and formatted labels
pivot_df = fig3_data.pivot(index='SectorGroup', columns='Year', values='CO2eq').fillna(0)
pivot_df['diff'] = pivot_df[2035] - pivot_df[2024]
pivot_df['label'] = pivot_df.index + ' (' + pivot_df['diff'].round(0).astype(int).astype(str) + ')'
label_map = pivot_df['label'].to_dict()

# Step 4: Map new labels to fig3_data
fig3_data['SectorLabel'] = fig3_data['SectorGroup'].map(label_map)

# Step 5: Sort by absolute difference
ordered_labels = fig3_data[fig3_data['Year'] == 2035][['SectorGroup', 'SectorLabel']]
ordered_labels['abs_diff'] = ordered_labels['SectorGroup'].map(pivot_df['diff'].abs())
ordered_labels = ordered_labels.drop_duplicates().sort_values('abs_diff')
ordered_label_list = ordered_labels['SectorLabel'].tolist()

# Step 6: Plot as bar chart
fig3b = go.Figure()

# 2024: red bar
subset_2024 = fig3_data[fig3_data['Year'] == 2024]
fig3b.add_trace(go.Bar(
    x=subset_2024['CO2eq'],
    y=subset_2024['SectorLabel'],
    name='2024',
    orientation='h',
    marker_color='red'
))

# 2035: blue bar
subset_2035 = fig3_data[fig3_data['Year'] == 2035]
fig3b.add_trace(go.Bar(
    x=subset_2035['CO2eq'],
    y=subset_2035['SectorLabel'],
    name='2035',
    orientation='h',
    marker_color='blue'
))

# Step 7: Layout
fig3b.update_layout(
    barmode='group',
    title="Fig 3: NDC_BASE-RG Emissions by Aggregate Sector Group (Sorted by Δ 2035-2024)",
    xaxis_title="CO₂eq (kt)",
    yaxis_title="Sector Group (Δ 2035–2024)",
    yaxis=dict(categoryorder='array', categoryarray=ordered_label_list),
    template="simple_white",
    font=dict(family="Arial", size=14),
    margin=dict(l=60, r=30, t=60, b=100),
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5)
)


fig3b.show()
fig3b.write_image(f"{output_dir}/fig3b_ndc_emissions_by_sector.jpeg")
fig3b_data.to_csv(f"{output_dir}/fig3b_data.csv", index=False)



## Figure 4

In [82]:
df

,Process,Commodity,Year,Scenario,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,CO2eq,ScenarioFamily
138,SATIMMarginals,CF4,2024,NDC_CPP4-08-RG,Marginals,-3798.909553,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP4 Variant
139,SATIMMarginals,CF4,2024,NDC_CPP4-085-RG,Marginals,-2304.482119,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP4 Variant
140,SATIMMarginals,CF4,2024,NDC_CPP4-09-RG,Marginals,-1087.177196,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP4 Variant
141,SATIMMarginals,CF4,2024,NDC_CPP3-075-RG,Marginals,-5925.639034,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP3
142,SATIMMarginals,CF4,2024,NDC_CPP3-08-RG,Marginals,-3848.301145,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,Industry Process emissions CF4,CF4,CF4,-0.0,CPP3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20051291,XUPSOCR,UPSOCR,2035,NDC_CPP4EKH-RG,FlowOut,362.745208,NaN,NaN,Other,Supply Sector Oil Crude ...,NaN,NaN,NaN,NaN,Supply Sector Oil Crude ...,Crude Oil,0,0.0,CPP4 Variant
20051292,XUPSOCR,UPSOCR,2035,NDC_CPP4EKH-HG,FlowOut,362.745208,NaN,NaN,Other,Supply Sector Oil Crude ...,NaN,NaN,NaN,NaN,Supply Sector Oil Crude ...,Crude Oil,0,0.0,CPP4 Variant
20051293,XUPSOCR,UPSOCR,2035,NDC_CPP4GE-RG,FlowOut,362.745208,NaN,NaN,Other,Supply Sector Oil Crude ...,NaN,NaN,NaN,NaN,Supply Sector Oil Crude ...,Crude Oil,0,0.0,CPP4 Variant
20051294,XUPSOCR,UPSOCR,2035,NDC_CPP4GE-09-RG,FlowOut,362.745208,NaN,NaN,Other,Supply Sector Oil Crude ...,NaN,NaN,NaN,NaN,Supply Sector Oil Crude ...,Crude Oil,0,0.0,CPP4 Variant


In [63]:
df['IPCC_Category_L1'].unique()


array([nan, '2 Industrial Processes and Product Use ', '3 Agriculture',
       '4 LULUCF', '5 Waste', '1 Energy'], dtype=object)

In [75]:
# --- Fig 4: Stacked bar of emissions in 2035 by scenario and sector group ---

df_f4 = df[df['IPCC_Category_L1']=='1 Energy'] # keep only energy combustion emissions


# --- Fig 4: Stacked bar of emissions in 2035 by scenario and sector group ---
df_f4['SectorGroup'] = df_f4['Sector'].apply(map_sector_group)

# Step 1: Sum 2035 emissions by Scenario

# select a random sample of 18 scenarios from the middle of the sorted list of scenarios, excluding the one with the smallest and the largest total emissions.

total_2035 = df_f4[df_f4['Year'] == 2035].groupby('Scenario')['CO2eq'].sum().sort_values()
selected_scenarios = total_2035.head(1).index.tolist() + \
                     total_2035.iloc[1:-1].sample(n=18, random_state=42).index.tolist() + \
                     total_2035.tail(1).index.tolist()

# Step 2: Get 2035 emissions by SectorGroup and Scenario
fig4_data = df_f4[(df_f4['Year'] == 2035) & (df_f4['Scenario'].isin(selected_scenarios))]
fig4_grouped = fig4_data.groupby(['Scenario', 'SectorGroup'])['CO2eq'].sum().reset_index()

# Step 3: Add NDC_BASE-RG 2024 emissions as comparison
total_2024 = df_f4[(df_f4['Year'] == 2024) & (df_f4['Scenario'] == 'NDC_BASE-RG')].groupby(['SectorGroup'])['CO2eq'].sum().reset_index()
total_2024['Scenario'] = 'NDC_BASE-RG'

fig4_grouped = pd.concat([fig4_grouped, total_2024], ignore_index=True)

# Step 4: Order scenarios by total emissions
scenario_order = fig4_grouped.groupby('Scenario')['CO2eq'].sum().sort_values().index.tolist()

# Step 5: Pivot for stacked bar chart
fig4_pivot = fig4_grouped.pivot(index='Scenario', columns='SectorGroup', values='CO2eq').fillna(0)
fig4_pivot = fig4_pivot.reindex(scenario_order)

# Step 6: Plot stacked bar
fig4 = go.Figure()
sector_order = ['All others', 'Refineries', 'Transport', 'Industry', 'Power']
for sector in sector_order:
    if sector in fig4_pivot.columns:
        fig4.add_trace(go.Bar(
            x=fig4_pivot.index,
            y=fig4_pivot[sector],
            name=sector
        ))

fig4.update_layout(
    barmode='stack',
    title="Energy Emissions",
    xaxis_title="",
    yaxis_title="CO₂eq (kt)",
    template="simple_white",
    font=dict(family="Arial", size=14),
    margin=dict(l=60, r=30, t=60, b=100),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

fig4.add_trace(go.Scatter(
    x=['NDC_BASE-RG'],
    y=[fig4_pivot.loc['NDC_BASE-RG'].sum()],
    mode='text',
    text=['2024'],
    textposition='top center',
    showlegend=False,
    textfont=dict(size=12, color='black', family='Arial Black')
))

fig4.show()
fig4.write_image(f"{output_dir}/fig4_2035_StackedBarByScenarios.jpeg")
fig4_grouped.to_csv(f"{output_dir}/fig4_data.csv", index=False)

C:\Users\01425453\AppData\Local\Temp\ipykernel_47340\2413063992.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Figure 5: measures impact on BASE

In [ ]:

# Filter the data
fig5_data = df[
    (df['ScenarioFamily'] == 'BASE') &
    (df['EconomicGrowth'] == 'Reference')
]

# Aggregate emissions by Scenario and Year
fig5_grouped = fig5_data.groupby(['Scenario', 'Year'])['CO2eq'].sum().reset_index()

# Create line chart
fig5 = go.Figure()

for scenario in fig5_grouped['Scenario'].unique():
    subset = fig5_grouped[fig5_grouped['Scenario'] == scenario]
    fig5.add_trace(go.Scatter(
        x=subset['Year'],
        y=subset['CO2eq'],
        mode='lines',
        name=scenario,
        line=dict(width=2)
    ))

# Apply layout
fig5.update_layout(
    title="Fig 5: Total Emissions for BASE Scenarios (Reference Economic Growth)",
    xaxis_title="Year",
    yaxis_title="CO₂eq (kt)",
    template="simple_white",
    font=dict(family="Arial", size=14),
    margin=dict(l=60, r=30, t=60, b=60),
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5)
)

fig5.show()

C:\Users\01425453\AppData\Local\Temp\ipykernel_47340\375530059.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:

# # some results for joseph to look at for pulp and paper

# scen_list= [ 'NDC_BASE-RG',
#             'NDC_BASE-08-RG',
#             'NDC_CPP1-RG',
#             'NDC_CPP4-RG',
#             'NDC_CPP4-08-HG'
# ]


# df_forJM = report_df[report_df['Scenario'].isin(scen_list)]

# df_forJM['Scenario'].unique()

# #write

# df_forJM.to_csv("C:/Models/REPORT00_forJoe.csv")